In [5]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('static/population(seoul).csv')  # 파일명을 적절히 변경해주세요

# 서울특별시 데이터만 필터링
df = df[df['시도명'] == '서울특별시']

# 성별과 연령대별로 데이터 재구성을 위한 함수
def reshape_data(row, gender):
    base_cols = ['행정기관코드', '기준연월', '시도명', '시군구명']
    age_cols = [f'{i}세{gender}' for i in range(0, 110)] + [f'110세이상 {gender}']
    
    data = row[base_cols].tolist() + [gender] + row[age_cols].tolist()
    return pd.Series(data, index=base_cols + ['성별'] + list(range(111)))

# 남성과 여성 데이터 재구성
male_df = df.apply(lambda row: reshape_data(row, '남자'), axis=1)
female_df = df.apply(lambda row: reshape_data(row, '여자'), axis=1)

# 재구성된 데이터 합치기
reshaped_df = pd.concat([male_df, female_df], ignore_index=True)

# 연령대 그룹화 함수
def age_group(age):
    if age < 10:
        return '0-9세'
    elif age < 20:
        return '10대'
    elif age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    elif age < 50:
        return '40대'
    elif age < 60:
        return '50대'
    else:
        return '60대이상'

# 연령대별 인구수 계산 및 데이터 재구성
final_data = []
for _, row in reshaped_df.iterrows():
    age_groups = {'0-9세': 0, '10대': 0, '20대': 0, '30대': 0, '40대': 0, '50대': 0, '60대이상': 0}
    for i in range(111):
        group = age_group(i)
        age_groups[group] += row[i]
    
    for group, population in age_groups.items():
        final_data.append({
            '시도명': row['시도명'],
            '시군구명': row['시군구명'],
            '성별': row['성별'],
            '연령대': group,
            '인구수': population
        })

# 최종 DataFrame 생성
final_df = pd.DataFrame(final_data)

# 결과 출력
print(final_df.tail(10))

     시도명 시군구명  성별    연령대   인구수
0  서울특별시  종로구  남자   0-9세   295
1  서울특별시  종로구  남자    10대   536
2  서울특별시  종로구  남자    20대   674
3  서울특별시  종로구  남자    30대   670
4  서울특별시  종로구  남자    40대   802
5  서울특별시  종로구  남자    50대   859
6  서울특별시  종로구  남자  60대이상  1282
7  서울특별시  종로구  남자   0-9세   223
8  서울특별시  종로구  남자    10대   262
9  서울특별시  종로구  남자    20대   423
